### Import the necessary libraries

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
import tensorflow as tf
import csv
import numpy as np
import pandas as pd
from PIL import Image

In [ ]:
from keras import Model
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from keras.utils import Sequence
from keras.backend import epsilon

### Load the training data from train.csv file

In [0]:
trainlabels = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Colab Notebooks/train_labels.csv')
trainlabels.head(5)

### Print the shape of the train dataset

In [0]:
trainlabels.shape

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Colab Notebooks')

### Declare a variable IMAGE_SIZE = 128 as we will be using MobileNet which will be taking Input shape as 128 * 128 

In [0]:
# UsingMobileNet takes images of size 128*128*3
IMAGE_SIZE = 128  
EPOCHS = 10
BATCH_SIZE = 32

DATASET_FOLDER = "/content/drive/My Drive/Colab Notebooks/"
TRAIN_CSV = DATASET_FOLDER+"train_labels.csv"

images_zip_path = DATASET_FOLDER + "images.zip"

### With the help of csvr.reader write a for loop which can load the train.csv file and store the path, width, height, x0,y0,x1,y1 in induvidual variables. <br>
1. Create a list variable known as 'path' which has all the path for all the training images
2. Create an array 'coords' which has the resized coordinates of the bounding box for the training images

<u>Note:</u> All the training images should be downsampled to 128 * 128 as it is the input shape of MobileNet (which we will be using for Object detection). Hence the corresponding coordinates of the bounding boxes should be changed to match the image dimension of 128 * 128 

In [0]:
import csv
with open(TRAIN_CSV, 'r') as csvfile:
    paths = []
    coords = np.zeros((sum(1 for line in csvfile), 4))
    coords = np.delete(coords, (0), axis=0)
    reader = csv.reader(csvfile, delimiter=',')
    csvfile.seek(0)
    next(reader)
    
    for col, row in enumerate(reader):

        for i, r in enumerate(row[1:3]): # Parse row with seven entities
            row[i+1] = int(r)
            
        for i, r in enumerate(row[4:8]): # Parse row with seven entities
            row[i+4] = int(r)
            
        path, image_height, image_width, cls, x0, y0, x1, y1 = row
        path = "/content/drive/My Drive/Colab Notebooks/images" + "/" + path.split('/')[-1]
        coords[col, 0] = x0 * IMAGE_SIZE / image_width
        coords[col, 1] = y0 * IMAGE_SIZE / image_height 
        coords[col, 2] = (x1 - x0) * IMAGE_SIZE / image_width
        coords[col, 3] = (y1 - y0) * IMAGE_SIZE / image_height
        paths.append(path)

batch_images = np.zeros((len(paths), IMAGE_SIZE, IMAGE_SIZE, 3), dtype=np.float32)
for i, f in enumerate(paths):
    img = Image.open(f) # Read image
    print(i)
    img = img.resize((IMAGE_SIZE, IMAGE_SIZE)) # Resize image
    img = img.convert('RGB')
    batch_images[i] = preprocess_input(np.array(img, dtype=np.float32))
                 

In [0]:
len(batch_images)

### Write a for loop which can load all the training images into a variable 'batch_images' using the paths from the 'paths' variable
<u>Note:</u> Convert the image to RGB scale as the MobileNet accepts 3 channels as inputs   

### Import MobileNet and load MobileNet into a variable named 'model' which takes input shape of 128 * 128 * 3. Freeze all the layers. Add convolution and reshape layers at the end to ensure the output is 4 coordinates

In [0]:
model = MobileNet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, alpha=ALPHA)
for layer in model.layers:
    layer.trainable = False
x = model.layers[-1].output
x = Conv2D(4, kernel_size=4, name="coords")(x)
x = Reshape((4,))(x)
model = Model(inputs=model.input, outputs=x)

### Define a custom loss function IoU which calculates Intersection Over Union

In [0]:
def loss(gt,pred):
    intersections = 0
    unions = 0
    # To identify the minimum and maximum of the square boundries to find the intersection
    diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
    diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
    intersection = diff_width * diff_height
    
    # Compute union
    area_gt = gt[:,2] * gt[:,3]
    area_pred = pred[:,2] * pred[:,3]
    union = area_gt + area_pred - intersection

#     Compute intersection and union over multiple boxes
    for j, _ in enumerate(union):
        if union[j] > 0 and intersection[j] > 0 and union[j] >= intersection[j]:
            intersections += intersection[j]
            unions += union[j]

    # Compute IOU. Use epsilon to prevent division by zero
    iou = np.round(intersections / (unions + epsilon()), 4)
    iou = iou.astype(np.float32)
    return iou

def IoU(y_true, y_pred):
    iou = tf.py_func(loss, [y_true, y_pred], tf.float32)
    return iou

In [ ]:
model.summary()

### Write model.compile function & model.fit function with: <br>
1. Optimizer = Adam, Loss = 'mse' and metrics = IoU
2. Epochs = 30, batch_size = 32, verbose = 1

In [0]:
gt = coords
model.compile(optimizer='Adam', loss='mse', metrics=[IoU])
model.fit(batch_images,gt,epochs=20,batch_size = 32,verbose=1)

### Pick a test image from the given data

In [0]:
# Pick a test image, run model, show image, and show predicted bounding box overlaid on the image
import cv2
filename = '/content/drive/My Drive/Colab Notebooks/images/raccoon-10.jpg'
unscaled = cv2.imread(filename) # Original image for display

In [0]:
image_height, image_width, _ = unscaled.shape
image = cv2.resize(unscaled, (IMAGE_SIZE, IMAGE_SIZE)) # Rescaled image to run the network
feat_scaled = preprocess_input(np.array(image, dtype=np.float32))

### Resize the image to 128 * 128 and preprocess the image for the MobileNet model

In [0]:
region = model.predict(x=np.array([feat_scaled]))[0] # Predict the BBox

### Predict the coordinates of the bounding box for the given test image

In [0]:
x0 = int(region[0] * image_width / IMAGE_SIZE) # Scale the BBox
y0 = int(region[1] * image_height / IMAGE_SIZE)

x1 = int((region[0] + region[2]) * image_width / IMAGE_SIZE)
y1 = int((region[1] + region[3]) * image_height / IMAGE_SIZE)

In [0]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

### Plot the test image using .imshow and draw a boundary box around the image with the coordinates obtained from the model

In [0]:
# Create figure and axes
fig,ax = plt.subplots(1)

# Display the image
ax.imshow(unscaled)

# Create a Rectangle patch
rect = patches.Rectangle((x0, y0), x1 - x0, y1 - y0, linewidth=2, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

## Problem 2 : Time Series Analysis using LSTM

### Download Data
Link: https://datamarket.com/data/set/2324/daily-minimum-temperatures-in-melbourne-australia-1981-1990#!ds=2324&display=line

#### Description
Daily minimum temperatures in Melbourne, Australia, 1981-1990

Units: Degrees Celcius

#### Steps before loading
- Rename the column name with temperature values to "Temperature"
- In the last, there is one extra row in the data, remove it by opening the file and save it again.
- There are some values in Temperature column which have a "?" before them, they will give error, remove "?" before them and save the file
- If you don't want to do these steps, just load the data file given by Great Learning.

### Import required libraries

In [0]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.layers import LSTM
from keras  import callbacks
from keras import optimizers
import pandas as pd 
import tensorflow as tf
import numpy as np

#### Mount google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Load the  data file

In [0]:

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/daily-minimum-temperatures-in-me.csv')
df.head(5)

### Plot the data

In [0]:
df.plot()

In [0]:
df.describe()

### Check for null values and treat them

In [0]:
# Null Values?
df.isnull().sum()

#### Drop null values (if any)

In [0]:
df.info

### Get the representation of the distribution of data in the form of histogram

In [0]:
df.hist(bins=10)

### Check the maximum and minimum values

In [0]:
print('Min', np.min(df))
print('Max', np.max(df))

### Normalize the data

In [0]:
#Normalization is not required as the temperature is in one scale

### Check the maximum and minimum values of scaled data

In [0]:
datasetconv = df[['Temperature']].astype('float32')

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(datasetconv)

In [0]:
print('Min', np.min(scaled))
print('Max', np.max(scaled))

### Split data into Training and Testing set 
<u>Note</u> The train and test data has to be sequential. Keep the first 70% of data in training and the bottom 30% in test data 

In [0]:
#Create the model
train_size = int(len(scaled) * 0.70)
test_size = len(scaled - train_size)
train, test = scaled[0:train_size, :], scaled[train_size: len(scaled), :]
print('train: {}\ntest: {}'.format(len(train), len(test)))

## Create the sequential data
Map the temperature at a particular time t to the temperature at time t+n, where n is any number you define.

For example: to map temperatures of consecutive days, use t+1, i.e. loop_back = 1

#### Define your function to create dataset

In [0]:
def create_dataset(dataset, look_back=1):
    print(len(dataset), look_back)
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        print(i)
        print('X {} to {}'.format(i, i+look_back))
        print(a)
        print('Y {}'.format(i + look_back))
        print(dataset[i + look_back, 0])
        dataset[i + look_back, 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

### Use function to get training and test set

In [0]:
look_back = 1
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

### Transform the prepared train and test input data into the expected structure using numpy.reshape()

In [0]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_train.shape)
print(X_test.shape)

### Define sequntial model, add LSTM layer and compile the model

In [0]:
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=200, batch_size=batch_size, verbose=2, shuffle=True)

### Train the model for 100 epochs with batch size = 1

In [0]:
#Train the model for 100 epochs with batch size = 1
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=batch_size, verbose=2, shuffle=True)

### Make Predictions and Evaluate your model

In [0]:
#def Predict(Score, df=1):
    #print(len(dataset),)
    #X = []